In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from rich.console import Console
from rich.table import Table
from IPython.display import display, HTML

# --- Visual Styling ---
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.float_format", "{:,.2f}".format)
console = Console()

# Standard Color Palette
COLOR_PASS = "#2E8B57"  # SeaGreen
COLOR_FAIL = "#CD5C5C"  # IndianRed
COLOR_WARN = "#FF8C00"  # DarkOrange


def display_header(text, subtext=""):
    """Orion Standard Header"""
    console.rule(f"[bold cyan]{text}")
    if subtext:
        console.print(f"[italic dim]{subtext}[/]", justify="center")


In [2]:
# ==========================================
# ⚙️ CONFIGURATION
# ==========================================
AUDIT_FILE_PATH = "TLC_Universal_Audit_Report_Raw.csv"  # <--- CHANGE THIS PATH
# ==========================================

# Load Data
try:
    df = pd.read_csv(AUDIT_FILE_PATH)

    # Handle Date Parsing (Audit script produces 'audit_month' usually YYYY-MM-DD)
    if "audit_month" in df.columns:
        df["audit_month"] = pd.to_datetime(df["audit_month"])
        df = df.sort_values("audit_month")
        TIME_COL = "audit_month"
    else:
        # Fallback if column name differs
        TIME_COL = df.columns[0]

    print(f"✅ Successfully loaded: {AUDIT_FILE_PATH}")
    print(f"📅 Range: {df[TIME_COL].min().date()} to {df[TIME_COL].max().date()}")
    print(f"📊 Months: {len(df)}")
    print(f"🚕 Total Rows Audited: {df['total_rows'].sum():,.0f}")

except FileNotFoundError:
    print(f"❌ Error: File not found at {AUDIT_FILE_PATH}")


✅ Successfully loaded: TLC_Universal_Audit_Report_Raw.csv
📅 Range: 2019-02-01 to 2025-09-01
📊 Months: 80
🚕 Total Rows Audited: 1,416,267,643


<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="paradox-check" style="display: none;">1. The Paradox Check (Physics & Logic)</h1>
<!-- VISIBLE H1 -->
<h1 id="paradox-check-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">1. The Paradox Check (Physics & Logic)</h1>

In [3]:
# Identify Paradox Columns dynamically
paradox_cols = [c for c in df.columns if "paradox" in c]

if not paradox_cols:
    print("✅ No Paradox Columns found (Data might be too raw or pre-cleaned).")
else:
    # Summary Table
    p_summary = df[paradox_cols].sum().sort_values(ascending=False).to_frame(name="Total Failures")
    p_summary["% Failure"] = (p_summary["Total Failures"] / df["total_rows"].sum()) * 100

    # Styling
    def highlight_fail(val):
        color = "red" if val > 0 else "green"
        return f"color: {color}; font-weight: bold"

    display(p_summary.style.applymap(highlight_fail).format({"Total Failures": "{:,.0f}", "% Failure": "{:.6f}%"}))

    # Visualization over time
    if p_summary["Total Failures"].sum() > 0:
        fig = px.line(
            df,
            x=TIME_COL,
            y=paradox_cols,
            title="<b>Paradox Violations Over Time</b><br><i>(Spikes indicate bad data ingestion or corruption)</i>",
            markers=True,
        )
        fig.update_layout(hovermode="x unified", height=400)
        fig.show()
    else:
        console.print("[bold green]✨ ZERO PARADOXES DETECTED. PHYSICS INTEGRITY: 100%[/]")


C:\Users\yopro\AppData\Local\Temp\ipykernel_21072\551555953.py:16: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(p_summary.style.applymap(highlight_fail).format({"Total Failures": "{:,.0f}", "% Failure": "{:.6f}%"}))


,Total Failures,% Failure
paradox_slave_labor_count,"30,060,994",2.122550%
paradox_time_travel_count,"49,631",0.003504%
paradox_teleport_count,"4,411",0.000311%


<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="health-matrix" style="display: none;">2. The Health Matrix (Nulls, Zeros, Negatives)</h1>
<!-- VISIBLE H1 -->
<h1 id="health-matrix-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">2. The Health Matrix (Nulls, Zeros, Negatives)</h1>

In [4]:
# Extract Quality Columns
quality_cols = [c for c in df.columns if c.endswith(("_nulls", "_zeros", "_negatives"))]

if quality_cols:
    # Aggregate totals
    q_total = df[quality_cols].sum().reset_index()
    q_total.columns = ["Metric_Full", "Count"]

    # Parse Feature Name and Issue Type
    q_total["Feature"] = q_total["Metric_Full"].apply(lambda x: x.rsplit("_", 1)[0])
    q_total["Issue"] = q_total["Metric_Full"].apply(lambda x: x.rsplit("_", 1)[1])

    # Pivot
    q_pivot = q_total.pivot(index="Feature", columns="Issue", values="Count").fillna(0)

    # Calculate %
    total_global = df["total_rows"].sum()
    for col in ["nulls", "zeros", "negatives"]:
        if col in q_pivot.columns:
            q_pivot[f"{col}_%"] = (q_pivot[col] / total_global) * 100

    # Order columns
    display_cols = []
    for issue in ["nulls", "zeros", "negatives"]:
        if issue in q_pivot.columns:
            display_cols.extend([issue, f"{issue}_%"])

    # Display Heatmap
    display(
        q_pivot[display_cols]
        .style.background_gradient(cmap="Reds", subset=[c for c in display_cols if "%" in c])
        .format("{:,.0f}", subset=[c for c in display_cols if "%" not in c])
        .format("{:.4f}%", subset=[c for c in display_cols if "%" in c])
    )
else:
    print("No health metrics found in report.")


Issue,nulls,nulls_%,zeros,zeros_%,negatives,negatives_%
Feature,,,,,,
airport_fee,"414,435,054",29.2625%,"922,542,597",65.1390%,0,0.0000%
base_passenger_fare,0,0.0000%,"3,756,344",0.2652%,"1,288,592",0.0910%
bcf,0,0.0000%,"89,529,428",6.3215%,0,0.0000%
cbd_congestion_fee,0,0.0000%,"118,441,817",8.3630%,0,0.0000%
congestion_surcharge,"513,041",0.0362%,"864,542,588",61.0437%,0,0.0000%
driver_pay,0,0.0000%,"36,494,777",2.5768%,"21,744",0.0015%
sales_tax,0,0.0000%,"47,684,127",3.3669%,3,0.0000%
speed_kmh,0,0.0000%,"859,438",0.0607%,0,0.0000%
tips,0,0.0000%,"1,170,038,323",82.6142%,0,0.0000%


<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="dist-atlas" style="display: none;">3. The Distribution Atlas (Statistical Deep Dive)</h1>
<!-- VISIBLE H1 -->
<h1 id="dist-atlas-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">3. The Distribution Atlas (Statistical Deep Dive)</h1>

In [ ]:
import warnings
# ==========================================
# 3. THE DISTRIBUTION ATLAS
# ==========================================
display_header("3. The Distribution Atlas (Statistical Deep Dive)")

# 1. Dynamic Feature Detection
all_cols = df.columns.tolist()
potential_features = set()
for c in all_cols:
    if c.endswith("_mean"):
        potential_features.add(c.replace("_mean", ""))

sorted_features = sorted(list(potential_features))


# 2. Display Function
def show_feature_stats(feature_name):
    # Filter columns
    target_suffixes = ["_min", "_p01", "_p50", "_mean", "_std", "_p99", "_p99.9", "_max"]
    selected_cols = []
    for suffix in target_suffixes:
        col = f"{feature_name}{suffix}"
        if col in df.columns:
            selected_cols.append(col)

    if not selected_cols:
        return

    # Create mini dataframe
    cols_to_show = [TIME_COL, "total_rows"] + selected_cols
    temp = df[cols_to_show].copy()

    # Rename columns (Remove prefix, Upper Case)
    rename_map = {c: c.replace(f"{feature_name}_", "").upper() for c in selected_cols}
    temp = temp.rename(columns=rename_map)

    # --- Define Style Groups ---
    # 1. Low/Safe Distribution -> Blue Gradient
    cols_grad_blue = [c for c in ["MIN", "P01", "P50"] if c in temp.columns]

    # 2. Magnitude/Average -> Blue Bar
    cols_bar_blue = [c for c in ["MEAN"] if c in temp.columns]

    # 3. High/Risk Distribution -> Red Gradient
    cols_grad_red = [c for c in ["STD", "P99", "P99.9"] if c in temp.columns]

    # 4. Maximums -> Red Bar
    cols_bar_red = [c for c in ["MAX"] if c in temp.columns]

    # Formatting
    format_cols = cols_grad_blue + cols_bar_blue + cols_grad_red + cols_bar_red

    print(f"\n🔹 STATISTICS FOR: {feature_name.upper()}")

    # Suppress "Divide by Zero" warnings for columns with constant values (like 0.0 fees)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        styler = (
            temp.style.background_gradient(cmap="Blues", subset=cols_grad_blue)
            .bar(subset=cols_bar_blue, color="#5DA5DA", height=70, width=80)
            .background_gradient(cmap="Reds", subset=cols_grad_red)
            .bar(subset=cols_bar_red, color="#D9534F", height=70, width=80)
            .format({TIME_COL: "{:%b %Y}"})
            .format("{:,.2f}", subset=format_cols)
            .format("{:,.0f}", subset=["total_rows"])
        )

        display(styler)


# 3. Execute Loop
for feat in sorted_features:
    show_feature_stats(feat)


──────────────────────────────── 3. The Distribution Atlas (Statistical Deep Dive) ────────────────────────────────


🔹 STATISTICS FOR: AIRPORT_FEE


,audit_month,total_rows,MIN,P01,P50,MEAN,STD,P99,P99.9,MAX
0,Feb 2019,"20,159,102",nan,nan,nan,nan,nan,nan,nan,nan
1,Mar 2019,"23,864,598",nan,nan,nan,nan,nan,nan,nan,nan
2,Apr 2019,"21,734,822",nan,nan,nan,nan,nan,nan,nan,nan
3,May 2019,"22,329,247",nan,nan,nan,nan,nan,nan,nan,nan
4,Jun 2019,"21,001,990",nan,nan,nan,nan,nan,nan,nan,nan
5,Jul 2019,"20,303,312",0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00
6,Aug 2019,"20,126,113",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Sep 2019,"20,069,321",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,Oct 2019,"21,162,290",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Nov 2019,"21,635,568",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00



🔹 STATISTICS FOR: BASE_PASSENGER_FARE


,audit_month,total_rows,MIN,P01,P50,MEAN,STD,P99,P99.9,MAX
0,Feb 2019,"20,159,102",-163.28,1.91,10.74,15.71,16.13,78.39,152.45,"1,097.29"
1,Mar 2019,"23,864,598",-149.02,2.35,10.18,15.43,16.28,79.55,155.39,"2,255.49"
2,Apr 2019,"21,734,822",-111.21,2.20,11.00,16.25,16.70,82.04,157.07,"1,313.41"
3,May 2019,"22,329,247",-118.90,2.87,12.06,17.74,17.94,89.45,168.27,"2,371.27"
4,Jun 2019,"21,001,990",-179.32,3.32,12.72,18.54,18.39,91.70,171.09,"1,642.24"
5,Jul 2019,"20,303,312","-1,969.59",0.00,11.64,17.10,17.97,87.09,169.95,"2,215.22"
6,Aug 2019,"20,126,113","-1,513.59",0.00,10.89,15.85,17.80,83.50,165.69,"2,710.50"
7,Sep 2019,"20,069,321",-82.46,2.94,12.70,18.29,17.76,88.43,164.35,"1,505.83"
8,Oct 2019,"21,162,290",-133.83,2.81,12.87,18.18,17.35,86.31,162.66,"2,419.64"
9,Nov 2019,"21,635,568",-81.14,2.77,11.97,17.20,16.94,84.12,159.34,"1,156.26"


...

,audit_month,total_rows,MIN,P01,P50,MEAN,STD,P99,P99.9,MAX
0,Feb 2019,"20,159,102",0.00,0.77,4.63,7.50,8.72,40.76,79.03,755.20
1,Mar 2019,"23,864,598",0.00,0.77,4.68,7.56,8.72,40.76,79.57,728.31
2,Apr 2019,"21,734,822",0.00,0.77,4.70,7.59,8.75,41.07,80.40,667.07
3,May 2019,"22,329,247",0.00,0.77,4.65,7.64,8.89,41.68,81.45,672.37
4,Jun 2019,"21,001,990",0.00,0.79,4.67,7.66,8.94,41.94,82.68,753.41
5,Jul 2019,"20,303,312",0.00,0.79,4.62,7.60,9.00,41.70,84.58,846.77
6,Aug 2019,"20,126,113",0.00,0.80,4.76,7.82,9.24,42.36,86.67,989.76
7,Sep 2019,"20,069,321",0.00,0.77,4.75,7.81,9.09,42.73,82.54,683.55
8,Oct 2019,"21,162,290",0.00,0.77,4.63,7.72,9.02,42.76,81.79,676.42
9,Nov 2019,"21,635,568",0.00,0.76,4.55,7.57,8.83,41.94,80.21,608.51



🔹 STATISTICS FOR: TRIP_MILES


,audit_month,total_rows,MIN,P01,P50,MEAN,STD,P99,P99.9,MAX
0,Feb 2019,"20,159,102",0.00,0.48,2.88,4.66,5.42,25.33,49.11,469.26
1,Mar 2019,"23,864,598",0.00,0.48,2.91,4.70,5.42,25.33,49.44,452.55
2,Apr 2019,"21,734,822",0.00,0.48,2.92,4.71,5.44,25.52,49.96,414.50
3,May 2019,"22,329,247",0.00,0.48,2.89,4.75,5.52,25.90,50.61,417.79
4,Jun 2019,"21,001,990",0.00,0.49,2.90,4.76,5.56,26.06,51.37,468.15
5,Jul 2019,"20,303,312",0.00,0.49,2.87,4.72,5.59,25.91,52.56,526.16
6,Aug 2019,"20,126,113",0.00,0.50,2.96,4.86,5.74,26.32,53.86,615.01
7,Sep 2019,"20,069,321",0.00,0.48,2.95,4.85,5.65,26.55,51.29,424.74
8,Oct 2019,"21,162,290",0.00,0.48,2.88,4.79,5.61,26.57,50.82,420.31
9,Nov 2019,"21,635,568",0.00,0.47,2.83,4.70,5.49,26.06,49.84,378.11



🔹 STATISTICS FOR: TRIP_TIME


,audit_month,total_rows,MIN,P01,P50,MEAN,STD,P99,P99.9,MAX
0,Feb 2019,"20,159,102",0.00,179.00,928.00,"1,117.97",773.06,"3,705.00","5,655.00","83,847.00"
1,Mar 2019,"23,864,598",0.00,180.00,932.00,"1,131.39",793.92,"3,840.00","5,820.00","82,950.00"
2,Apr 2019,"21,734,822",0.00,180.00,928.00,"1,133.24",804.76,"3,900.00","5,970.00","84,250.00"
3,May 2019,"22,329,247",0.00,187.00,945.00,"1,177.31",872.63,"4,286.00","6,572.00","82,061.00"
4,Jun 2019,"21,001,990",0.00,193.00,944.00,"1,174.86",871.94,"4,312.00","6,710.00","85,103.00"
5,Jul 2019,"20,303,312",0.00,189.00,906.00,"1,123.45",830.46,"4,052.00","6,609.00","81,525.00"
6,Aug 2019,"20,126,113",0.00,191.00,917.00,"1,131.88",828.90,"4,032.00","6,489.00","78,663.00"
7,Sep 2019,"20,069,321",0.00,192.00,962.00,"1,190.75",878.65,"4,273.00","6,602.00","78,053.00"
8,Oct 2019,"21,162,290",0.00,193.00,960.00,"1,181.01",856.40,"4,153.00","6,366.00","70,492.00"
9,Nov 2019,"21,635,568",0.00,190.00,933.00,"1,147.28",823.71,"4,082.00","6,183.00","86,053.00"


<!-- HIDDEN H1 FOR OUTLINE VIEW -->
<h1 id="cat-flags" style="display: none;">4. Categorical & Boolean Analysis</h1>
<!-- VISIBLE H1 -->
<h1 id="cat-flags-visible" style="font-family: 'Roboto Condensed', 'Arial Narrow', sans-serif; color: white; font-size: 22px; font-weight: bold; background-color: #0771A4; border-radius: 4px; padding: 12px 0px 12px 15px; margin-top: 20px;">4. Categorical & Boolean Analysis</h1>

In [6]:
# 1. Boolean Flags (Columns ending in _count_true)
flag_cols = [c for c in df.columns if c.endswith("_count_true")]

if flag_cols:
    display_header("Boolean Flag Prevalence")

    # Calculate % for visualization
    flag_df = df[[TIME_COL, "total_rows"] + flag_cols].copy()
    plot_cols = []

    for c in flag_cols:
        short_name = c.replace("_count_true", "")
        pct_col = f"{short_name} (%)"
        flag_df[pct_col] = (flag_df[c] / flag_df["total_rows"]) * 100
        plot_cols.append(pct_col)

    # Plot
    fig = px.line(flag_df, x=TIME_COL, y=plot_cols, title="<b>Boolean Flags over Time (%)</b>", markers=True)
    fig.update_layout(hovermode="x unified", height=450, yaxis_title="Percentage of Trips")
    fig.show()

# 2. Categorical Null Checks
# If we tracked nulls for categoricals (like weather_state_nulls)
cat_nulls = [c for c in df.columns if c.endswith("_nulls") and c.replace("_nulls", "") not in sorted_features]

if cat_nulls:
    display_header("Categorical Data Completeness")
    cat_df = df[cat_nulls].sum().to_frame(name="Total Missing")
    cat_df["% Missing"] = (cat_df["Total Missing"] / df["total_rows"].sum()) * 100
    display(cat_df[cat_df["Total Missing"] > 0].style.format({"Total Missing": "{:,.0f}", "% Missing": "{:.4f}%"}))
